# 120. User-Based Collaborative Filtering (협업 필터링)

### User-Based CF 의 알고리즘

| USER | M1 | M2 | M3 | M4 | M5 | U1과의 유사도 |
| --- | --- |--- |--- | ---|--- | --- |
| U1 | 2 | 5 | 3 |   |   |
|U2|4|4|3|5|1|0.19|
|U3|1|5|4| | 5| 0.89|
|U4|3|5|3|2|5  |0.94|
|U5|4|5|3|4| |0.65|
|U3, U4 평균|2|5|3.5|2|5|

1) U1과 취향이 비슷한 사용자를 찾는다. 취향은 각 사용자의 영화에 대한 평가의 유사성을 계산하여 찾는다. --> U3와 U4가 U1과 가장 높은 상관 관계를 보임.  

2) U1 과 가장 유사한 U3와 U4가 가장 좋게 평가한 영화를 찾는다. U1 이 아직 보지 않은 영화 M4, M5 에 대해 U3, U4 의 평점 평균을 내면 각각 2 와 5 이다. 따라서 평점 평균이 높은 M5를 U1도 좋아할 것으로 예상.

### MovieLens dataset
- MovieLens 라는 테스트 추천 시스템으로부터 수집된 데이터  
- 사용자들이 영화에 대해서 평가한 데이터  
- MovieLens에서는 사용자가 각 영화를 1점(최악)에서 5점(최고) 사이의 점수로 평가  

In [1]:
import sklearn
sklearn.__version__

'1.2.2'

In [2]:
# 필요한 라이브러리 임포트
import numpy as np 
import pandas as pd  
from sklearn.model_selection import train_test_split  

# from sklearn.metrics import root_mean_squared_error   # sklearn 최신 버전
from sklearn.metrics import mean_squared_error  # sklearn 구 버전

# 두 벡터 간의 코사인 유사도를 계산하기 위한 함수
from sklearn.metrics.pairwise import cosine_similarity

# 영화 데이터의 열 이름을 정의
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']

# 영화 (특성) 정보 로드
movies = pd.read_csv('data/u.item', sep='|', names=i_cols, encoding='latin-1')

movies.head(3)

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


- content-based filtering 과 달리 CF 는 similarity 를 이용하므로 **item의 feature 정보가 필요하지 않기 때문**에 movies dataframe에서 title 을 제외한 모든 다른 열을 drop 한다.  

In [3]:
# item의 특성 정보가 필요하지 않으므로 movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [4]:
# 사용자의 영화 평점(rating) 정보 로드
# 평점 데이터의 열 이름을 정의
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
# 평점 데이터를 파일에서 읽어옴
ratings = pd.read_csv('data/u.rating', sep='\t', names=r_cols, encoding='latin-1')  

# 로드된 평점 데이터의 첫 5개 행을 출력하여 데이터 확인
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


- ratings 만을 입력으로 사용

In [5]:
# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [6]:
# ratings를 train, test set 으로 분리
# train set으로 평점 matrix 생성
# test set으로 모델 성능 평가   
X = ratings.copy()

X_train, X_test = train_test_split(X, test_size=0.25, random_state=0)
X_train.shape, X_test.shape

((75000, 3), (25000, 3))

### train 데이터로 평점 matrix 생성

In [7]:
# 평점 매트릭스를 생성 - 사용자별로 각 영화에 대한 평점을 나타내는 2차원 테이블.
# 사용자 ID가 행 인덱스로, 영화 ID가 열 인덱스로 설정됩니다. 각 셀에는 해당 사용자가 해당 영화에 부여한 평점이 저장됩니다.
rating_matrix = X_train.pivot(index="user_id", columns="movie_id", values="rating")

print(rating_matrix.shape)  # 차원 (사용자수, 영화수)
rating_matrix.head(3)

(943, 1628)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1668,1669,1672,1673,1674,1676,1677,1678,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,NaN,5.0,NaN,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### User-Based CF 알고리즘

step 1) 모든 사용자간의 평가의 유사도를 계산  
step 2) 현재 추천 대상이 되는 사용자와 다른 사용자간의 유사도를 추출  
step 3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값 계산. 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자의 유사도로 가중 평균  
step 4) 아이템 중에서 예상 평가값이 가장 높은 N 개의 아이템을 추천  

이 알고리즘이 작동하기 위해서는 추천 대상 사용자가 평가한 아이템과 다른 사용자들이 평가한 동일한 아이템이 유사하게 계산되면 신뢰할 수 있는 모델이 만들어진 것이고, 이 모델을 이용하여 추천 대상 사용자가 보지 않은 아이템(영화)의 평점을 계산하고 가장 예상 평점이 높은 N 개 아이템을 추천하면 된다.    

이때, 추천 대상 사용자와 가장 유사도가 높은 K 개의 다른 사용자의 평가만 반영하여 예측 평가의 품질을 높인다.

이를 위해 추천 사용자, 영화 id 를 parameter 로 받아 예상 평점을 반환하는 함수를 작성

### step 1) 모든 사용자간의 평가의 유사도를 계산

In [8]:
# 평점 매트릭스의 복사본을 생성하고, NaN 값을 0으로 채웁니다. (평점을 부여하지 않은 영화는 평점이 0인 것으로 처리)
matrix_dummy = rating_matrix.copy().fillna(0)
print(matrix_dummy.shape)

# 사용자 간의 코사인 유사도를 계산합니다.
# 여기서 matrix_dummy의 각 행은 사용자를 나타내며, 열은 영화에 대한 사용자의 평점을 나타냅니다.
# cosine_similarity 함수는 이 행렬을 입력으로 받아, 모든 가능한 사용자 쌍 간의 코사인 유사도를 계산합니다.
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
print(user_similarity.shape)

# 코사인 유사도 매트릭스를 DataFrame으로 변환하고, 인덱스와 컬럼 이름을 rating_matrix의 인덱스(사용자 ID)로 설정합니다.
# 따라서, 이 DataFrame의 각 셀은 특정 사용자 쌍 사이의 코사인 유사도를 나타냅니다.
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity.head(10)

(943, 1628)
(943, 943)


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.140887,0.036981,0.069143,0.306388,0.302852,0.339506,0.241018,0.083494,0.296784,...,0.295321,0.104003,0.197249,0.155246,0.152691,0.080099,0.263132,0.089671,0.141146,0.331283
2,0.140887,1.000000,0.097246,0.181611,0.048586,0.160920,0.092840,0.080473,0.129376,0.133846,...,0.084098,0.162309,0.274986,0.288405,0.161569,0.114472,0.208187,0.079739,0.149775,0.118609
3,0.036981,0.097246,1.000000,0.211377,0.030859,0.038523,0.055345,0.006273,0.057778,0.037376,...,0.044604,0.061175,0.113719,0.054296,0.078177,0.021812,0.120308,0.091164,0.126480,0.037371
4,0.069143,0.181611,0.211377,1.000000,0.039629,0.056862,0.077217,0.125925,0.115134,0.038943,...,0.062547,0.045149,0.124937,0.163932,0.127203,0.000000,0.194381,0.115340,0.145046,0.056738
5,0.306388,0.048586,0.030859,0.039629,1.000000,0.187204,0.304344,0.163389,0.066394,0.172922,...,0.296144,0.097635,0.094838,0.095795,0.115138,0.040104,0.233320,0.083141,0.098550,0.284926
6,0.302852,0.160920,0.038523,0.056862,0.187204,1.000000,0.342748,0.119575,0.158508,0.456196,...,0.254693,0.092580,0.128484,0.193585,0.086407,0.106194,0.224265,0.085547,0.186713,0.199063
7,0.339506,0.092840,0.055345,0.077217,0.304344,0.342748,1.000000,0.250819,0.123244,0.390354,...,0.370937,0.089552,0.118486,0.092902,0.125924,0.074990,0.299872,0.044938,0.179860,0.293402
8,0.241018,0.080473,0.006273,0.125925,0.163389,0.119575,0.250819,1.000000,0.033990,0.195535,...,0.206664,0.028226,0.059777,0.030746,0.078015,0.037741,0.203665,0.027041,0.109983,0.203222
9,0.083494,0.129376,0.057778,0.115134,0.066394,0.158508,0.123244,0.033990,1.000000,0.191314,...,0.058054,0.000000,0.105988,0.096758,0.092140,0.052143,0.119705,0.177459,0.043415,0.025525


### 추천 대상이 되는 사용자에게 보지 않은 영화를 추천할 때 사용자가 부여할 rating 예측

1. 추천 대상 영화에 대해 이미 평가를 내린 모든 사용자의 rating값 가져오기 --> 영화 ID를 기준으로 모든 사용자의 평점 값을 필터링  
3. 추천 대상 영화를 평가하지 않은 사용자의 index 가져오기
4. 대상 영화를 평가하지 않은 사용자의 rating (null) 제거 --> 추천 대상 영화를 평가한 사용자만 남김

### step2) 현재 추천 대상이 되는 사용자와 다른 사용자들의 유사도를 추출

In [9]:
# 사용자별 영화 평점 테이블
rating_matrix[:3]

movie_id,1,2,3,4,5,6,7,8,9,10,...,1668,1669,1672,1673,1674,1676,1677,1678,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,NaN,5.0,NaN,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- `ratings_matrix(사용자별로 각 영화에 대한 평점을 나타내는 테이블)`에서 user_id 1 이 movie_id 5를 평가하지 않았으므로 user_id 1 에게 movie_id 5을 추천할 때의 예상 rating을 예측해 봅니다.

In [10]:
 # 현재 추천 대상이 되는 사용자
user_id = 1    
# 추천대상 영화
movie_id = 5 

#step2)현재 추천 대상이 되는 사용자와 다른 사용자들 간의 similarity 가져오기
sim_scores = user_similarity[user_id]
print(sim_scores.shape)
sim_scores.head()

(943,)


user_id
1    1.000000
2    0.140887
3    0.036981
4    0.069143
5    0.306388
Name: 1, dtype: float64

In [11]:
# step 3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값 계산. 
# 1.추천대상 영화(movie_id)에 대한 모든 사용자의 평점을 구함
# rating_matrix에서 영화 ID가 movie_id인 열을 선택
movie_ratings = rating_matrix[movie_id]

movie_ratings.head()

user_id
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: 5, dtype: float64

In [12]:
# 2.추천대상 영화를 평가하지 않은 사용자(movie_ratings에서 평점이 NaN)의 index 가져오기
non_rating_idx = movie_ratings[movie_ratings.isnull()].index

# 추천대상 영화에 대해 평점을 부여하지 않은 사용자의 수
print(non_rating_idx.shape)

# 평점을 부여하지 않은 사용자의 인덱스 목록 출력
non_rating_idx

(883,)


Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       934, 935, 936, 937, 938, 939, 940, 941, 942, 943],
      dtype='int64', name='user_id', length=883)

In [13]:
# 3. 추천대상 영화를 평가하지 않은 사용자의 rating (null) 제거 --> 추천 대상 영화를 평가한 사용자만 남김
movie_ratings = movie_ratings.drop(non_rating_idx)

# 추천대상 영화에 평점을 부여한 사용자의 수
print(movie_ratings.shape)

movie_ratings.head()

(60,)


user_id
13    1.0
21    2.0
28    3.0
43    4.0
44    4.0
Name: 5, dtype: float64

In [14]:
# 4. 현재 영화를 평가하지 않은 사용자의 similarity값 제거
sim_scores = sim_scores.drop(non_rating_idx)

#추천대상 영화에 실제로 평점을 부여한 사용자들 간의 유사도 값의 수
print(sim_scores.shape)

sim_scores.head()

(60,)


user_id
13    0.345234
21    0.135169
28    0.233203
43    0.284757
44    0.335388
Name: 1, dtype: float64

In [15]:
# 정규화: 유사도 점수의 합이 1이 되도록 조정
sim_scores /= sim_scores.sum()  

### 가장 유사도가 높은 상위 K명의 사용자 찾기

In [17]:
# step 4) 아이템 중에서 예상 평가값이 가장 높은 N 개의 아이템을 추천
n_neighbors = 10

# 5.유사도가 높은 사용자 순으로 정렬
top_users = sim_scores.sort_values(ascending=False).head(n_neighbors)
top_users

user_id
864    0.025806
916    0.025691
417    0.024804
308    0.024639
886    0.023720
682    0.022850
303    0.022844
378    0.022776
293    0.022590
109    0.022506
Name: 1, dtype: float64

In [18]:
# 6.상위 n_neighbors에 해당하는 평점만 추출
top_ratings = movie_ratings.loc[top_users.index]
top_ratings

user_id
864    4.0
916    3.0
417    4.0
308    4.0
886    3.0
682    3.0
303    2.0
378    3.0
293    3.0
109    3.0
Name: 5, dtype: float64

### Implicit Rating을 이용한 추천 공식

$$\tilde{r}_{ui} = \frac{\sum_{v \in KNN(u)}r_{vi} \cdot S_{vu}}{\sum_{v \in KNN(u)}S_{vu}}$$

In [19]:
# 7. 가중평균 계산
# top_users: 상위 유사도를 가진 사용자들의 유사도 점수 배열
# top_ratings: 상위 유사도를 가진 사용자들이 해당 영화에 매긴 평점 배열
# np.dot(top_users, top_ratings): 유사도 점수와 평점의 가중합을 계산
# top_users.sum(): 유사도 점수의 합으로 가중합을 나누어 가중평균을 계산
predicted_rating = np.dot(top_users, top_ratings) / top_users.sum()

# 계산된 가중평균 평점 출력
predicted_rating

3.219983653460719

###  특정 사용자에게 추천하는 영화(movie_id)의  가중평균 rating을 계산하는 함수
- 위의 과정을 model 함수로 작성
- 위에서 개별 사용자와 아이템에 대해 구하던 predicted_rating을 함수로 작성
- 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)

In [20]:
def UBCF_knn_model(user_id, movie_id, rating_matrix, user_similarity, n_neighbors=0):
    """
    사용자 기반 협업 필터링 모델 (k-최근접 이웃 알고리즘 사용)
    
    Parameters:
    user_id (int): 평점을 예측할 대상 사용자의 ID.
    movie_id (int): 평점을 예측할 대상 영화의 ID.
    rating_matrix (pd.DataFrame): 사용자-영화 평점 행렬. 행은 사용자, 열은 영화를 나타냄.
    user_similarity (pd.DataFrame): 사용자 간의 유사도 행렬. 행과 열은 각각 사용자 ID를 나타냄.
    n_neighbors (int, optional): 고려할 이웃의 수. 기본값은 0이며, 이 경우 모든 사용자를 고려함.
    
    Returns:
    float: 예측된 평점.
    """
    # 현재 영화가 rating_matrix의 컬럼에 있는지 확인
    if movie_id in rating_matrix.columns:  
        # 추천 대상 사용자와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 1. 추천 대상이 되는 영화에 대한 모든 사용자 평가
        movie_ratings = rating_matrix[movie_id]
        # 2. 추천 대상 영화를 평가하지 않은 사용자의 index 가져오기
        non_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 3. 추천 대상 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(non_rating_idx)
        # 4. 추천 대상 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        
        # 유사도의 합으로 나누기 전에 유사도가 0인지 확인
        if sim_scores.sum() > 0:
            sim_scores /= sim_scores.sum()
        else:
            return 3.0  # 유사도의 합이 0이라면 평균 평점을 반환
            
        # k가 지정되지 않은 경우 모든 유사도를 사용
        if n_neighbors == 0:  
            n_neighbors = len(sim_scores)
        
        # n_neighbors가 1 이상이면, 유사도가 높은 순으로 정렬하고 상위 n_neighbors만 사용
        if n_neighbors > 1:
            # 5.유사도가 높은 사용자 순으로 정렬
            top_users = sim_scores.sort_values(ascending=False).head(n_neighbors)
            # 6.상위 n_neighbors에 해당하는 평점만 추출
            top_ratings = movie_ratings.loc[top_users.index]
            # 7.가중평균 계산
            predicted_rating = np.dot(top_users, top_ratings) / top_users.sum()
        else:
            # 유사한 사용자가 없거나 한 명뿐인 경우 기본 평점을 할당
            predicted_rating = 3.0
    else:
        predicted_rating = 3.0  # 영화 ID가 rating_matrix에 없으면 기본 평점을 반환
    return predicted_rating

In [21]:
UBCF_knn_model(12, 203, rating_matrix, user_similarity, n_neighbors=30)

3.7939035147860083

In [22]:
UBCF_knn_model(166, 346, rating_matrix, user_similarity, n_neighbors=30)

2.950531782191326

In [23]:
UBCF_knn_model(244, 51, rating_matrix, user_similarity, n_neighbors=30)

3.5419635457654626

### 모델 성능 평가 - 모델별 RMSE를 계산하는 함수 
- Test set을 이용하여 모델 성능 측정  

In [24]:
def score(model, n_neighbors=0):
    # X_test 데이터셋에서 사용자 ID와 영화 ID의 쌍을 생성합니다.
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])

    # 생성된 각 ID 쌍에 대해 모델을 사용하여 예상 평점을 계산합니다.
    y_pred = np.array([model(user, movie, rating_matrix, user_similarity, n_neighbors) 
                       for (user, movie) in id_pairs])
    
    # X_test 데이터셋에서 실제 평점을 가져옵니다.
    y_true = np.array(X_test['rating'])
    
    # 실제 평점과 예측 평점 사이의 RMSE를 계산하여 반환합니다.
    #return root_mean_squared_error(y_true, y_pred)  # sklearn latest version
    return mean_squared_error(y_true, y_pred, squared=False)  # sklearn old version

In [25]:
# 정확도 계산
print("knn 감안하지 않은 simple CF - 모든 다른 사용자를 포함")
print(score(UBCF_knn_model, n_neighbors=0))

K = 30
print(f"knn 감안한 CF - {K} 개의 가장 유사한 사용자만 포함")
print(score(UBCF_knn_model, n_neighbors=K))

knn 감안하지 않은 simple CF - 모든 다른 사용자를 포함
1.023699962224089
knn 감안한 CF - 30 개의 가장 유사한 사용자만 포함
1.0200919120771834


### 특정 사용자에 대하여 영화 추천
- 이미 본 영화는 제외하고 보지 않은 영화 중에서 예상 평점이 높은 영화 추천

In [26]:
# rating matrix에서 추천 대상 사용자의 해당 row 가져오기
user_movie_ratings = rating_matrix.loc[user_id].copy()
user_movie_ratings

movie_id
1       5.0
2       3.0
3       4.0
4       3.0
5       NaN
       ... 
1676    NaN
1677    NaN
1678    NaN
1681    NaN
1682    NaN
Name: 1, Length: 1628, dtype: float64

In [27]:
# 가져온 row에서 추천 대상 사용자의 평점이 없는(보지 않은) 영화를 식별합니다.
unrated_movies = user_movie_ratings[user_movie_ratings.isnull()]

In [28]:
# 평가되지 않은 각 영화에 대해 예상 평점을 계산합니다.
predicted_ratings = {movie: UBCF_knn_model(user_id, movie, rating_matrix, user_similarity, n_neighbors=30)
                         for movie in unrated_movies.index}

# 예상 평점순으로 정렬된 Series를 생성합니다.
predicted_ratings_df = pd.Series(predicted_ratings).sort_values(ascending=False)[:5]
print(predicted_ratings_df.shape)

predicted_ratings_df

(5,)


1306    5.0
1450    5.0
1629    5.0
1617    5.0
814     5.0
dtype: float64

영화를 높은 예상 평점에 따라 제목을 뽑아 줌

In [29]:
recom_movies = movies.loc[predicted_ratings_df.index]
recommendations = recom_movies['title']
recommendations

1306        Carmen Miranda: Bananas Is My Business (1994)
1450                         Foreign Correspondent (1940)
1629    Silence of the Palace, The (Saimt el Qusur) (1...
1617                              King of New York (1990)
814                                   One Fine Day (1996)
Name: title, dtype: object

### 주어진 사용자에 대하여 추천 받는 함수
위 코드들을 하나의 함수로 작성 

In [30]:
def recom_movie(user_id, n_items, n_neighbors=30):
    """
    user_id: 추천을 받을 사용자의 ID. 이 ID는 rating_matrix에서 해당 사용자의 데이터에 접근하는 데 사용.
    n_items: 추천할 영화의 개수.
    n_neighbors: 이웃의 수
    """
    # rating matrix에서 추천 대상 사용자의 해당 row 가져오기
    user_movie_ratings = rating_matrix.loc[user_id].copy()

    # 가져온 row에서 추천 대상 사용자의 평점이 없는(보지 않은) 영화를 식별합니다.
    unrated_movies = user_movie_ratings[user_movie_ratings.isnull()]

    # 평가되지 않은 각 영화에 대해 예상 평점을 계산합니다.
    predicted_ratings = {movie: UBCF_knn_model(user_id, movie, rating_matrix, user_similarity, n_neighbors)
                         for movie in unrated_movies.index}

    # 예상 평점순으로 정렬된 Series를 생성합니다.
    predicted_ratings_df = pd.Series(predicted_ratings).sort_values(ascending=False)[:n_items]

    # 추천 영화의 제목을 가져옵니다.
    recom_movies = movies.loc[predicted_ratings_df.index]['title']

    return recom_movies

In [31]:
# 특정 사용자에 대해 n_items 개의 영화 추천
user_id = 2
n_items = 5
recom_movie(user_id, n_items, n_neighbors=30)

1656                                        Target (1995)
1629    Silence of the Palace, The (Saimt el Qusur) (1...
1306        Carmen Miranda: Bananas Is My Business (1994)
1617                              King of New York (1990)
1536                                          Cosi (1996)
Name: title, dtype: object

### 최적의 neighbor size 구하기

In [32]:
# 다양한 이웃 크기(neighbor size)에 대해 RMSE 계산 및 출력
for neighbor_size in [10, 20, 30, 40, 50, 60]:  
    print(f"Neighbor size = {neighbor_size} : RMSE = {score(UBCF_knn_model, neighbor_size):.4f}")

Neighbor size = 10 : RMSE = 1.0367
Neighbor size = 20 : RMSE = 1.0233
Neighbor size = 30 : RMSE = 1.0201
Neighbor size = 40 : RMSE = 1.0197
Neighbor size = 50 : RMSE = 1.0203
Neighbor size = 60 : RMSE = 1.0210
